Análisis Exploratorio de Datos (EDA)

In [1]:
#leer requirements para instalar las librerías
import pandas as pd
import plotly.express as px
import re
import nbformat as nbf

In [2]:
#leer el DataFrame
pokemon = pd.read_csv(r'C:\Users\Fer Titla\Desktop\TripleTen\sprint_7_tripleten\pokemon.csv')
pokemon.head()#leer DF completo
#pokemon[['name', 'japanese_name', 'attack']].head(10)#leer 10 primeras filas del DF

,abilities,against_bug,against_dark,against_dragon,against_electric,against_fairy,against_fight,against_fire,against_flying,against_ghost,...,percentage_male,pokedex_number,sp_attack,sp_defense,speed,type1,type2,weight_kg,generation,is_legendary
0,"['Overgrow', 'Chlorophyll']",1.0,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,1,65,65,45,grass,poison,6.9,1,0
1,"['Overgrow', 'Chlorophyll']",1.0,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,2,80,80,60,grass,poison,13.0,1,0
2,"['Overgrow', 'Chlorophyll']",1.0,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,3,122,120,80,grass,poison,100.0,1,0
3,"['Blaze', 'Solar Power']",0.5,1.0,1.0,1.0,0.5,1.0,0.5,1.0,1.0,...,88.1,4,60,50,65,fire,NaN,8.5,1,0
4,"['Blaze', 'Solar Power']",0.5,1.0,1.0,1.0,0.5,1.0,0.5,1.0,1.0,...,88.1,5,80,65,80,fire,NaN,19.0,1,0


In [3]:
#obtener información general del DF
pokemon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 801 entries, 0 to 800
Data columns (total 41 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   abilities          801 non-null    object 
 1   against_bug        801 non-null    float64
 2   against_dark       801 non-null    float64
 3   against_dragon     801 non-null    float64
 4   against_electric   801 non-null    float64
 5   against_fairy      801 non-null    float64
 6   against_fight      801 non-null    float64
 7   against_fire       801 non-null    float64
 8   against_flying     801 non-null    float64
 9   against_ghost      801 non-null    float64
 10  against_grass      801 non-null    float64
 11  against_ground     801 non-null    float64
 12  against_ice        801 non-null    float64
 13  against_normal     801 non-null    float64
 14  against_poison     801 non-null    float64
 15  against_psychic    801 non-null    float64
 16  against_rock       801 non

In [4]:
 #código para limpiar el DF
def clean_dataset(pokemon: pd.DataFrame) -> pd.DataFrame:
    """
    Limpia el DataFrame aplicando las siguientes reglas:
    - Nombres de columnas en minúsculas
    - Espacios reemplazados por '_'
    - Eliminación de caracteres especiales
    - Limpieza de valores tipo string
    """

    # =========================
    # 1. LIMPIEZA DE COLUMNAS
    # =========================
    pokemon.columns = (
        pokemon.columns
        .str.lower()
        .str.strip()
        .str.replace(' ', '_')
        .str.replace(r'[^a-z0-9_]', '', regex=True)
    )

    # =========================
    # 2. LIMPIEZA DE VALORES STRING
    # =========================
    text_columns = pokemon.select_dtypes(include='object').columns

    for col in text_columns:
        pokemon[col] = (
            pokemon[col]
            .str.lower()
            .str.strip()
            .str.replace(r'[^a-z0-9_ ]', '', regex=True)
        )

    return pokemon


In [5]:
# Observar la cantidad y porcentaje de los datos nulos 
columnas_nulos = pokemon.isnull().sum()
total_filas = len(pokemon)

for columna, cantidad in columnas_nulos[columnas_nulos > 0].items():
    porcentaje = (cantidad / total_filas) * 100
    print(
        f"Los NaN de '{columna}' son: {cantidad} "
        f"y representan el {porcentaje:.1f}% del total del DataFrame"
    )

Los NaN de 'height_m' son: 20 y representan el 2.5% del total del DataFrame
Los NaN de 'percentage_male' son: 98 y representan el 12.2% del total del DataFrame
Los NaN de 'type2' son: 384 y representan el 47.9% del total del DataFrame
Los NaN de 'weight_kg' son: 20 y representan el 2.5% del total del DataFrame


Métrica del 1% al 5%.  
Si los datos no representan ni el 1%, es mejor eliminarlos.  
Si representan más del 1%, pero menos de 5%, plantearse qué tanto afecta eliminar los datos.

In [6]:
#Todos los procesos de NaN están pensados en cómo se aplicarán 
#ya trabajando en streamlit

#Procesaré la columna type2 sustituyendo los NaN por la palabra unknown
#esa columna es The Secondary Type of the Pokemon
pokemon['type2'] = pokemon['type2'].fillna('unknown')
pokemon['type2'].isnull().sum()

#Procesaré la columna percentage_male sustituyendo los NaN por -1
#porque significan "genderless"
pokemon['percentage_male'] = pokemon['percentage_male'].fillna(-1)
pokemon['percentage_male'].isnull().sum()

#Procesaré la columna height_m imputandola con mediana segmentada
# Calcular la mediana de height_m (ignorando NaN)
mediana_height = pokemon['height_m'].median()

# Reemplazar los NaN con la mediana
pokemon['height_m'] = pokemon['height_m'].fillna(mediana_height)
pokemon['percentage_male'].isnull().sum()

#Imputaré los NaN de la columna weight_kg utilizando la mediana 
#para preservar la integridad de la distribución
# Calcular la mediana de weight_kg (ignorando NaN)
mediana_weight = pokemon['weight_kg'].median()

# Reemplazar los NaN con la mediana
pokemon['weight_kg'] = pokemon['weight_kg'].fillna(mediana_weight)
pokemon['weight_kg'].isnull().sum()


np.int64(0)

In [24]:
#Asegurarse de que capture_rate sea numericos para el gráfico de dispersión
print("Tipo de dato:", pokemon['capture_rate'].dtype)
print("Cantidad de NaN:", pokemon['capture_rate'].isna().sum())
pokemon['capture_rate'].unique()
#valor = "30 (Meteorite)255 (Core)"

#conteo = (pokemon['capture_rate'] == valor).sum()
#total = len(pokemon)

#porcentaje = (conteo / total) * 100

#print(f"'{valor}' aparece {conteo} veces y representa el {porcentaje:.2f}% del total del DF")

Tipo de dato: object
Cantidad de NaN: 0


array(['45', '255', '120', '127', '90', '190', '75', '235', '150', '25',
       '170', '50', '200', '100', '180', '60', '225', '30', '35', '3',
       '65', '70', '125', '205', '155', '145', '130', '140', '15', '220',
       '160', '80', '55', '30 (Meteorite)255 (Core)'], dtype=object)

El valor "30 (Meteorite)255 (Core)":

no es numérico

rompe cálculos y visualizaciones

no representa una categoría clara

Solo representa 0.12% del total del dataset

Eliminarlo no afecta la distribución general

Simplifica el análisis y evita errores posteriores

In [32]:
#codigo para eliminar el valor str y convertir la columna a numero
pokemon['capture_rate'] = (
    pokemon['capture_rate']
    .astype(str)                 # fuerza a texto
    .str.extract(r'(\d+)')       # extrae solo números
    .astype(float)               # convierte a número
)
pokemon['capture_rate'].dtype

dtype('float64')

In [7]:
# Observemos 10 filas aleatorias para verificar los cambios
pokemon[['height_m', 'percentage_male', 'type2', 'weight_kg']].sample(10)

,height_m,percentage_male,type2,weight_kg
380,2.0,100.0,psychic,60.0
69,1.0,50.0,poison,6.4
65,0.8,75.4,unknown,19.5
298,1.0,50.0,unknown,97.0
403,0.9,50.0,unknown,30.5
349,6.2,50.0,unknown,162.0
800,1.0,-1.0,fairy,80.5
338,0.4,50.0,ground,1.9
111,1.9,50.0,rock,120.0
747,0.7,50.0,water,14.5


# Comienzo del análisis 

In [8]:
#Top 15 pokemon más fácil de capturar
# Seleccionar columnas de interés
columnas_interes = [
    'name',
    'capture_rate',
    'weight_kg',
    'height_m',
    'attack',
    'defense',
    'percentage_male',
    'type1',
    'type2'
]

# Filtrar, ordenar y seleccionar el top 15 más fáciles de capturar
top_15_faciles = (
    pokemon[columnas_interes]
    .dropna(subset=['capture_rate'])  # asegurar capture_rate válido
    .sort_values(by='capture_rate', ascending=False)
    .head(15)
    .reset_index(drop=True)
)

top_15_faciles


,name,capture_rate,weight_kg,height_m,attack,defense,percentage_male,type1,type2
0,Lycanroc,90,27.3,1.0,115,75,50.0,rock,unknown
1,Fearow,90,38.0,1.2,90,65,50.0,normal,flying
2,Wimpod,90,12.0,0.5,35,40,50.0,bug,water
3,Quagsire,90,75.0,1.4,85,85,50.0,water,ground
4,Sandslash,90,27.3,1.0,100,120,50.0,ground,ice
5,Ariados,90,33.5,1.1,90,70,50.0,bug,poison
6,Beheeyem,90,34.5,1.0,75,75,50.0,psychic,unknown
7,Lairon,90,120.0,0.9,90,140,50.0,steel,rock
8,Crobat,90,75.0,1.8,90,80,50.0,poison,flying
9,Medicham,90,31.5,1.3,100,85,50.0,fighting,psychic


In [9]:
#Top 15 de pokemon más dificles de capturar
# Columnas de interés
columnas_interes = [
    'name',
    'capture_rate',
    'weight_kg',
    'height_m',
    'attack',
    'defense',
    'percentage_male',
    'type1',
    'type2'
]

# Filtrar valores válidos y ordenar por dificultad
top_15_dificiles = (
    pokemon[columnas_interes]
    .dropna(subset=['capture_rate'])
    .sort_values(by='capture_rate', ascending=True)
    .head(15)
    .reset_index(drop=True)
)

top_15_dificiles

,name,capture_rate,weight_kg,height_m,attack,defense,percentage_male,type1,type2
0,Kadabra,100,56.5,1.3,35,30,75.4,psychic,unknown
1,Spiritomb,100,108.0,1.0,92,108,50.0,ghost,dark
2,Gothorita,100,18.0,0.7,45,70,24.6,psychic,unknown
3,Duosion,100,8.0,0.6,40,50,50.0,psychic,unknown
4,Hawlucha,100,21.5,0.8,92,75,50.0,fighting,flying
5,Araquanid,100,82.0,1.8,70,92,50.0,water,bug
6,Whirlipede,120,58.5,1.2,55,99,50.0,bug,poison
7,Ninjask,120,12.0,0.8,90,45,50.0,bug,flying
8,Swadloon,120,7.3,0.5,63,90,50.0,bug,grass
9,Palpitoad,120,17.0,0.8,65,55,50.0,water,ground


In [10]:
resumen = (
    comparacion
    .groupby('grupo')[[
        'weight_kg',
        'height_m',
        'attack',
        'defense',
        'percentage_male'
    ]]
    .mean()
    .round(2)
)

resumen


NameError: name 'comparacion' is not defined

## Generación de histograma

In [ ]:
#Análisis previo al histograma
# Columnas relevantes
columnas = [
    'name',
    'capture_rate',
    'weight_kg',
    'height_m',
    'attack',
    'defense',
    'percentage_male',
    'type1',
    'type2'
]

df_base = pokemon[columnas].dropna(subset=['capture_rate'])

# Top 15 fáciles
faciles = (
    df_base
    .sort_values(by='capture_rate', ascending=False)
    .head(15)
    .assign(grupo='Fácil de capturar')
)

# Top 15 difíciles
dificiles = (
    df_base
    .sort_values(by='capture_rate', ascending=True)
    .head(15)
    .assign(grupo='Difícil de capturar')
)

# Unir ambos
comparacion = pd.concat([faciles, dificiles], ignore_index=True)


In [ ]:
#creación del histograma
fig = px.histogram(
    comparacion,
    x='attack',
    color='grupo',
    nbins=10,
    title='Distribución de la Fuerza (Attack) — Fácil vs Difícil de Capturar',
    labels={
        'attack': 'Fuerza (Attack)',
        'grupo': 'Grupo',
        'count': 'Cantidad de Pokémon'
    },
    opacity=0.75,
    color_discrete_map={
        'Fácil de capturar': '#2ECC71',     # verde
        'Difícil de capturar': '#E74C3C'    # rojo
    }
)

fig.update_layout(
    bargap=0.15,
    title_x=0.5
)

fig.show()


Cómo se interpreta:

🟩 Verde a la izquierda/derecha → Pokémon fáciles con menor/mayor fuerza

🟥 Rojo concentrado → Pokémon difíciles con fuerza alta

Superposición → características compartidas

Separación clara → diferenciación estructural

## Generación de un gráfico de dispersión

In [ ]:
#Convertiremos los valores inválidos de capture_rate
cols_numericas = ['weight_kg', 'attack', 'capture_rate']

for col in cols_numericas:
    pokemon[col] = pd.to_numeric(pokemon[col], errors='coerce')

In [33]:
#generación de gráfico de dispersión
# Limpiar filas necesarias
df_scatter = pokemon.dropna(subset=['weight_kg', 'attack', 'type1'])

# Crear el gráfico de dispersión
fig = px.scatter(
    df_scatter,
    x='weight_kg',
    y='attack',
    color='type1',
    size='capture_rate',
    title='Relación entre Peso y Fuerza de los Pokémon',
    labels={
        'weight_kg': 'Peso (kg)',
        'attack': 'Fuerza (Attack)',
        'type1': 'Tipo',
        'capture_rate': 'Capture Rate'
    },
    opacity=0.7
)

# Ajustes visuales
fig.update_layout(
    title_x=0.5
)

fig.show()


Cómo leer este gráfico

Eje X (Peso): tamaño físico del Pokémon

Eje Y (Attack): fuerza ofensiva

Color: tipo principal → segmentación clara

Tamaño del punto: facilidad de captura

grande → fácil

pequeño → difícil